<img src="./auxdata/eurac_EO.png" alt="EURAC Institute for Earth Observation" title="EURAC Institute for Earth Observation" width="300"/>

In this notebook we will use openEO to generate a cloud free RGB composite based on Sentinel-2 data.

In [164]:
import openeo
import csv
import pandas as pd
from openeo.rest.datacube import PGNode, THIS
from openeo.processes import mean, eq, median, or_, array_element, clip
import xarray as xr
from pyproj import Proj, transform, Transformer, CRS

Tile of interest

In [9]:
tile_of_interest = "32TPS"

Get the UTM extent of the tile we are interested in, e.g. 32TPS

In [98]:
s2gridpath = './auxdata/tabularize_s2_footprint.csv'
s2grid = pd.read_csv(s2gridpath, sep=',')
tile_description = s2grid.loc[s2grid["tile_id"]==tile_of_interest]
epsg = tile_description['epsg'].item()

32632

Get the bounding box of the tile. Each UTM tile has an extent of 109800 meters along x and y.

In [11]:
ul_x = tile_description["ul_x"].values[0]
ul_y = tile_description["ul_y"].values[0]
ll_x = ul_x + 109800
ll_y = ul_y - 109800

In [99]:
points_x = [ul_x, ll_x]
points_y = [ul_y, ll_y]
transformer = Transformer.from_crs(epsg, 4326)
points_lat,points_lon = transformer.transform(points_x,points_y)

Connect to the EURAC openEO back-end

In [128]:
euracEndpoint = "https://openeo.eurac.edu"
conn = openeo.connect(euracEndpoint).authenticate_oidc(client_id="openEO_PKCE")

Authenticated using refresh token.


Set the temporal extent we want to consider

In [129]:
temporal_extent = ["2017-06-01","2017-07-01"]

Set the spatial extent of the tile based on the extracted bounds

In [130]:
spatial_extent = {'west':points_lon[0],'east':points_lon[1],'north':points_lat[0],'south':points_lat[1]}

Set the collection name for the Sentinel-2 data

In [131]:
S2_collection = 'S2_L1C_T32TPS'

Set the bands we want to load

In [135]:
bands = ['B02','B03','B04','FMASK']

Load the S2 data:

In [150]:
S2_data = conn.load_collection(S2_collection,temporal_extent=temporal_extent,bands=bands).filter_bbox(spatial_extent)

We need to mask out (setting it to not a number) the zero values, i.e. where there is no data.

In [159]:
data_mask = S2_data.filter_bands('B04').reduce_dimension(dimension="bands",reducer = lambda value: eq(array_element(value,0),0))
S2_L1C_masked = S2_data.mask(data_mask)

We create a mask for clouds (FMASK == 4) and cloud shadows (FMASK == 2).

In [151]:
cloud_mask = S2_data.filter_bands('FMASK').reduce_dimension(dimension="bands",reducer = lambda value: or_(eq(array_element(value,0),2),eq(array_element(value,0),4)))

Apply the mask to the S2 data

In [160]:
S2_L1C_masked = S2_L1C_masked.filter_bands(['B02','B03','B04']).mask(cloud_mask)

Compute the median over time

In [161]:
S2_data_masked_median = S2_L1C_masked.reduce_dimension(dimension='DATE',reducer=median)

Clip the data between 0 and 1800 for a better visualization

In [165]:
S2_data_masked_median_clipped = S2_data_masked_median.apply(lambda value: value.clip(0,1800))

Save the result as netCDF

In [166]:
S2_data_masked_median_nc = S2_data_masked_median_clipped.save_result(format='netCDF')

Start a batch job and wait until it is marked as finished

In [167]:
job_title = 'S2_L1C_RGB_cloud_free_composite4'
job = conn.create_job(S2_data_masked_median_nc,title=job_title)
job_id = job.job_id
if job_id:
    print('Batch job created with id: ',job_id)
    job.start_job()
else:
    print('Error! Job ID is None')

Batch job created with id:  26a41da1-d483-4ade-98b5-e0c13a753518


In [170]:
job = conn.job(job_id)
job

<RESTJob job_id='26a41da1-d483-4ade-98b5-e0c13a753518'>

In [76]:
results = job.get_results()
results.download_files('./results/')

[PosixPath('results/process.json'), PosixPath('results/result.nc')]